In [1]:
import os, re, sys, json, pandas
from io import StringIO
from collections import deque, OrderedDict
from Spanners.Treeify import Treeify

In [2]:
df = pandas.read_excel('data.xlsx').astype(str)
df

,A,B,C
0,1,2,4
1,1,2,5
2,1,3,6
3,1,3,7


In [9]:
class Branch:
	'''
	utility to recurse down into a parsed dataframe
	'''

	def __init__(self, dq=None, stack=None):
		'''
		class object used to create isolation between levels
		and copies the dataframe into a reduced set to remove global sharing issues
		'''
		self.dq = dq.copy()
		if stack:
			self.stack = stack
		else:
			self.stack = [ OrderedDict([]) ]

	def pushd(self, value):
		self.putd(value)
		self.stack.append(
			self.stack[-1][value][-1]
		)

	def putd(self, value):
		if value not in self.stack[-1].keys():
			self.stack[-1][value] = [ OrderedDict([]) ]

	def popd(self):
		self.stack.pop()

	def graft(self, indent=''):
		columns = deque(self.dq.columns)
		if len(columns) == 0: return
		column = columns.popleft()

		for value in self.dq[column].unique():
			print(f'{indent}{value}')

			self.pushd(value)

			dqq = self.dq[self.dq[column] == value][columns]
			Branch(dq=dqq, stack=self.stack).graft(indent=f'\t{indent}')

			self.popd()


In [10]:
tree = Branch(dq=df)

In [11]:
tree.graft()

1
	2
		4
		5
	3
		6
		7


In [12]:
tree.stack

[OrderedDict([('1',
               [OrderedDict([('2',
                              [OrderedDict([('4', [OrderedDict()]),
                                            ('5', [OrderedDict()])])]),
                             ('3',
                              [OrderedDict([('6', [OrderedDict()]),
                                            ('7', [OrderedDict()])])])])])])]

In [13]:
treeify = Treeify(
    ascii = False,
    colour = True,
)

treeify.process(StringIO(json.dumps(tree.stack)), sys.stdout)

[0]
 └─1
    └─[0]
       ├─2
       │  └─[0]
       │     ├─4
       │     │  └─[0]
       │     └─5
       │        └─[0]
       └─3
          └─[0]
             ├─6
             │  └─[0]
             └─7
                └─[0]